<a href="https://colab.research.google.com/github/luke-scot/emissions-tracking/blob/main/C_THRU_IHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# C-THRU IHS data
Notebook for interacting with IHS Markit data. Please modify and save your own copies depending on what you wish to do with the data but you must keep it within the C-THRU project. If you have any issues please contact Luke at lshc3@cam.ac.uk.

Currently access to first 1,000 products of a total of 8,790 due to download restrictions.

In [1]:
#@title Run this cell to set up notebook

%%capture
import gdown, os
import pyarrow
import pandas as pd
import ipywidgets as ipw
from google.colab import files
from ipywidgets.widgets.widget_layout import Layout

class IHS_data(object):
  def __init__(self) -> None:
      self.folder, self.zip = 'data', 'data.zip'
      if not os.path.exists(self.folder): 
        gdown.download('https://drive.google.com/uc?id=1jrJGRls3YfDNjp6V4ovJtoeX0W0IgmMu', self.zip ,quiet=True)
        ! unzip $self.zip -d $self.folder
        ! rm $self.zip
      self.products = pd.read_feather('data/products.feather')
      self.materials = pd.read_feather('data/materials.feather')
      self.properties = ['Name','Geography','Process']
      self.matProperties = ['Code', 'Type']

def filter_display(data, properties):
  sel = ipw.HBox([ipw.SelectMultiple(options=['Any']+list(data[prop].unique()), value=('Any',), rows=10,description=prop, disabled=False, layout=Layout(width='500px')) for prop in properties])
  display(sel)
  return sel

def display_data(data,selections,show=True):
  if type(selections) is not list: selections = [selections]
  for selects in selections:
    for num, sels in enumerate(selects.children):
      if sels.value != ('Any',):
        data = data[[i in list(sels.value) for i in data[sels.description].values]]
  if show: display(data)
  return data

IHS = IHS_data()

In [ ]:
# Product selection
selections = filter_display(IHS.products,IHS.properties)

In [ ]:
# Show products
products = display_data(IHS.products,selections)

In [ ]:
# Material selection
components = filter_display(IHS.materials,IHS.matProperties)

In [ ]:
# Show materials
materials = display_data(IHS.materials,[selections,components])

In [ ]:
# Save results to csv files
productFile, materialsFile = 'products.csv', 'materials.csv'
products.to_csv(productFile), materials.to_csv(materialsFile)
download = [files.download(f) for f in [productFile, materialsFile]]